# A 2-Hours Seminar about [InterCom](https://github.com/Tecnologias-multimedia/intercom)

## Contents

1. What is InterCom?
2. De-jittering through buffering.
3. Decreasing the bit-rate through compression.
4. A simple bit-rate control through quantization.
5. Removing intra-frame redundancy.
6. Removing intra-channel redundancy.
7. Future developments.

## 1. What is InterCom?
InterCom is a real-time intercommunicator (currently, only transmitting audio). The sequence of [frames](https://python-sounddevice.readthedocs.io/en/0.3.12/api.html) (two stereo samples) is splitted into chunks and each one is transmitted in an [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol) packet.

### A [minimal](https://tecnologias-multimedia.github.io/study_guide/minimal/) implementation

In [ ]:
# Interrupt with: Kernel -> Interrupt
!python ~/intercom/minimal.py --show_stats

## 2. De-[jitter](https://en.wikipedia.org/wiki/Jitter)ing through [buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/)

### Which is the expected Internet latency?

In [ ]:
!ping -c 10 8.8.8.8 # Google DNS

### Which is the latency and jitter in my host?

In [ ]:
!ping -c 10 localhost

Too good to do simulations using `localhost` :-/

### Which is the expected Internet latency?

In [ ]:
!ping -c 10 8.8.8.8 # Google DNS

### Let's increases latency and jitter for the `localhost` link
* Latency = 20 ms.
* Jitter = 5 ms.
* Correlation between RTTs = 0.25 (<1.0).
* Statistical distribution of the RTTs = normal.

In [ ]:
# Remember to add "your_username_here ALL=(ALL) NOPASSWD: ALL" to the end of /etc/sudoers.
# Notice that these times express RTTs, not simple one-way lantencies.
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root handle 1: netem delay 20ms 5ms 25% distribution normal
!tc qdisc show dev lo

### (Optional) Remove `tc` rule

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

### Let's see ... now

In [ ]:
!ping -c 10 localhost

Better for our simulations.

### Let's listen again to minimal

In [ ]:
!python ~/intercom/minimal.py --show_stats

Quite bad :-/ The chunks don't arrive with a constant cadence to the receiver :

![Buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/graphics/timelines.svg)

### And finallyl, let's hidden the jitter

In [ ]:
!python ~/intercom/buffer.py --show_stats # 100 ms is the default buffering time

Much better!

## 3. Decreasing the transmission bit-rate through compression

### Estimation of the [throughput](https://en.wikipedia.org/wiki/Throughput) of the `localhost` link
Notice that the following technique depends heavely on the packet size, that using ping is limited to 64 KB (including the IP header). For this reason, the results can be only approximated :-/

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo
!ping -c 1 -s 65527 localhost

In [ ]:
!printf "Gbps = "
!echo 65527*8/0.147/2/1000/1000 | bc -l

Too high! We need harder conditions for testing InterCom that without compression requires

\begin{equation}
\frac{44100\frac{\text{frames}}{\text{second}}\times 2\frac{\text{samples}}{\text{frame}}\times 2\frac{\text{bytes}}{\text{sample}}\times 8\frac{\text{bits}}{\text{byte}}}{1000} = 1411.2 ~\text{kbps}
\end{equation}

### Let's simulate (also) a throughput of 200 kbps

In [1]:
# Definitive rules for the rest of experiments
!sudo tc qdisc add dev lo root handle 1: netem delay 20ms 5ms 25% distribution normal
!sudo tc qdisc add dev lo parent 1:1 handle 10: tbf rate 200kbit burst 1024kbit limit 1024kbit

### (Optional) Remove the previous `tc` rules

In [ ]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

In [ ]:
!ping -c 1 -s 65507 localhost

In [ ]:
print("Throughput =", 65507*8/522/2, "kbps")

###  Insufficient throughput?
... or too many kbps?

In [ ]:
!python ~/intercom/buffer.py --show_stats

### [Compression](https://tecnologias-multimedia.github.io/study_guide/compress/) using [DEFLATE](https://en.wikipedia.org/wiki/DEFLATE)

The chunks can be compressed with [LZSS](https://en.wikipedia.org/wiki/Lempel-Ziv-Storer-Szymanski) (that is based on [LZ77](https://github.com/vicente-gonzalez-ruiz/LZ77)) and [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding) (see also [this](https://vicente-gonzalez-ruiz.github.io/Huffman_coding/)).

We have choosen this text compressor codec because:
1. It's fast.
2. Works well when repeated strings are found at the input.
3. I's available with [The Standard Python Library](https://docs.python.org/3/library/) ([zlib](https://docs.python.org/3/library/zlib.html)).

In [ ]:
!python ~/intercom/compress.py --show_stats

Notice that a lot of chunks are still lost. DEFLATE is not enough!

## 4. A simple [bit-rate "control"](https://tecnologias-multimedia.github.io/study_guide/quantization/) through [quantization](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb)

Quantization removes the less relevant information (mainly [electronic noise](https://en.wikipedia.org/wiki/Noise_(electronics)) ...) and helps to increase the [compression ratio](https://en.wikipedia.org/wiki/Compression_ratio). In lossy signal compression, [dead-zone quantizers](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb) are commonly used because they tend to generate 0's that like to entropy compressors.

In InterCom, we lossely "control" the bit-rate because a strict bit-rate control through the variation of the quantization step $\Delta$ is computationally intensive (we must determine the [Rate/Distortion curve](https://en.wikipedia.org/wiki/Rate%E2%80%93distortion_theory) of the current chunk to find $\Delta$ before to quantize, compress and send it) and it's hard to compute in real-time . For this reason, the current implementation estimates the number of lost chunks per second (in the last second) and use this information to increase or decrease the quantization step for the chunks (of the next second).

In [ ]:
!python ~/intercom/br_control.py --show_stats

## 5. [Spatial (inter-channel) decorrelation]((https://tecnologias-multimedia.github.io/study_guide/spatial_decorrelation/))
The samples of a (stereo) frame tend to have similar amplitudes. For this reason, we apply [Mid/Side stereo coding](https://en.wikipedia.org/wiki/Joint_encoding#M/S_stereo_coding) (before quantization).

In [ ]:
!python ~/intercom/intra_frame_decorrelation.py --show_stats

## 6. [Temporal (intra-channel) decorrelation](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/)
The samples of each channel exhibit temporal redundancy. Therefore, we use a [Discrete Wavelet Transform (DWT)](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/) to exploit it (before quantization).

In [ ]:
!python ~/my-intercom/intra_channel_decorrelation.py --show_stats

## 7. Future work
1. Currently, quantization minimizes the MSE (Mean Square Error). [Perceptual](https://en.wikipedia.org/wiki/Psychoacoustics) quantization should increase the compression ratio without increasing the perceived distortion.
2. Transmit [video](https://en.wikipedia.org/wiki/Video).